In [6]:
# importing libraries
import os
import time as T
import random
import tsfel

import pandas as pd
import numpy as np

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier

# groq
from groq import Groq
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"), # stored API key in virtual environment (not going to GitHub)
)

# Data Processing and Featurization

## MakeDataset

In [7]:
# Constants
time = 10
offset = 100
folders = ["LAYING","SITTING","STANDING","WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"]
classes = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}

combined_dir = os.path.join("Combined")

#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
                                                # Train Dataset
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

X_train=[]
y_train=[]
dataset_dir = os.path.join(combined_dir,"Train")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir,folder))

    for file in files:

        df = pd.read_csv(os.path.join(dataset_dir,folder,file),sep=",",header=0)
        df = df[offset:offset+time*50]
        X_train.append(df.values)
        y_train.append(classes[folder])

X_train = np.array(X_train)
y_train = np.array(y_train)


#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
                                                # Test Dataset
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

X_test=[]
y_test=[]
dataset_dir = os.path.join(combined_dir,"Test")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir,folder))
    for file in files:

        df = pd.read_csv(os.path.join(dataset_dir,folder,file),sep=",",header=0)
        df = df[offset:offset+time*50]
        X_test.append(df.values)
        y_test.append(classes[folder])

X_test = np.array(X_test)
y_test = np.array(y_test)

#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
                                                # Final Dataset
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

# USE THE BELOW GIVEN DATA FOR TRAINING and TESTING purposes

# concatenate the training and testing data
X = np.concatenate((X_train,X_test))
y = np.concatenate((y_train,y_test))

# split the data into training and testing sets. Change the seed value to obtain different random splits.
seed = 4
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=seed,stratify=y)

print("Training data shape: ",X_train.shape)
print("Testing data shape: ",X_test.shape)

#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

Training data shape:  (126, 500, 3)
Testing data shape:  (54, 500, 3)


## TSFEL

In [ ]:
# making a list of of pandas dataframes corresponding to each sample
X_train_dfs = [pd.DataFrame(sample, columns=['accx', 'accy', 'accz']) for sample in X_train]
X_test_dfs = [pd.DataFrame(sample, columns=['accx', 'accy', 'accz']) for sample in X_test]

X_train_dfs = [df.apply(lambda x: np.sqrt(x['accx']**2 + x['accy']**2 + x['accz']**2), axis=1) for df in X_train_dfs]
X_test_dfs = [df.apply(lambda x: np.sqrt(x['accx']**2 + x['accy']**2 + x['accz']**2), axis=1) for df in X_test_dfs]

# consider all features
cfg_file = tsfel.get_features_by_domain()  

# get list of feature vectors for each dataframe (or sample)           
# choosing `fs=50` because the data was sampled at 50Hz                              
X_train_tsfel_dfs = [tsfel.time_series_features_extractor(cfg_file, df, fs=50) for df in X_train_dfs]
X_train_tsfel = pd.concat(X_train_tsfel_dfs, axis=0).fillna(0).values

X_test_tsfel_dfs = [tsfel.time_series_features_extractor(cfg_file, df, fs=50) for df in X_test_dfs]
X_test_tsfel = pd.concat(X_test_tsfel_dfs, axis=0).fillna(0).values

# merge the list of dataframes into a single dataframe
col = X_train_tsfel_dfs[0].columns
X_train_tsfel_df = pd.DataFrame(X_train_tsfel, columns = col)
X_test_tsfel_df = pd.DataFrame(X_test_tsfel, columns = col)

# do the following for the training data and then choose remaining columns from the test data
# remove columns where the feature is constant throughout all samples
for col in X_train_tsfel_df.columns:
    if len(X_train_tsfel_df[col].unique()) == 1:
        X_train_tsfel_df.drop(col, axis=1, inplace=True)

# remove highly correlated features (columns) from the training data
corr = X_train_tsfel_df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
tri_df = corr.mask(mask)
to_drop = [c for c in tri_df.columns if any(tri_df[c] > 0.9)] # threshold = 0.9
X_train_tsfel_df.drop(columns=to_drop, inplace=True)

# remove the same columns from the test data
X_test_tsfel_df = X_test_tsfel_df[X_train_tsfel_df.columns] 

# Filter columns starting with '0_FFT mean coefficient_'
filtered_cols = [col for col in X_train_tsfel_df.columns if col.startswith('0_FFT mean coefficient_')]
# Calculate the sum of squares for the filtered columns to get energy
X_train_tsfel_df['0_Energy'] = X_train_tsfel_df[filtered_cols].pow(2).sum(axis=1)
X_test_tsfel_df['0_Energy'] = X_test_tsfel_df[filtered_cols].pow(2).sum(axis=1)
# Drop the filtered columns
X_train_tsfel_df.drop(filtered_cols, axis=1, inplace=True)
X_test_tsfel_df.drop(filtered_cols, axis=1, inplace=True)

# feature selection
features = ['0_Autocorrelation', '0_Centroid', '0_Energy', '0_Kurtosis', '0_Max power spectrum', '0_Mean', '0_Mean diff', '0_Median',
       '0_Median diff', '0_Neighbourhood peaks', '0_Power bandwidth',
       '0_Skewness', '0_Slope', '0_Spectral distance', '0_Spectral entropy',
       '0_Spectral positive turning points', '0_Spectral skewness',
       '0_Spectral variation', '0_Wavelet absolute mean_8',
       '0_Wavelet entropy', '0_Wavelet variance_8']

X_train_tsfel_df = X_train_tsfel_df[features]
X_test_tsfel_df = X_test_tsfel_df[features]

# Question 1

### Zero Shot vs Few Shot: A Qualitative Comparision

In [9]:
label = {1: "WALKING", 2: "WALKING_UPSTAIRS", 3: "WALKING_DOWNSTAIRS", 4: "SITTING", 5: "STANDING", 6: "LAYING"}

Testing the LLM with Zero Shot prompting on 5 random samples

In [36]:
X_test_tsfel_df.loc[5].to_dict()

{'0_Autocorrelation': 4.0,
 '0_Centroid': 4.985172868880233,
 '0_Energy': 0.04403701574107831,
 '0_Kurtosis': -0.5375661524976398,
 '0_Max power spectrum': 3.604770415927572,
 '0_Mean': 1.049793479716819,
 '0_Mean diff': 4.5496276621879624e-05,
 '0_Median': 1.0186349478511663,
 '0_Median diff': 0.0129247391665257,
 '0_Neighbourhood peaks': 17.0,
 '0_Power bandwidth': 5.1000000000000005,
 '0_Skewness': 0.2921059091342811,
 '0_Slope': 5.547106249216689e-06,
 '0_Spectral distance': -104143.19960980392,
 '0_Spectral entropy': 0.492597392267046,
 '0_Spectral positive turning points': 78.0,
 '0_Spectral skewness': 2.0376811420807743,
 '0_Spectral variation': 0.8573145527149457,
 '0_Wavelet absolute mean_8': 0.09370415452419341,
 '0_Wavelet entropy': 2.0598362590465467,
 '0_Wavelet variance_8': 0.512597258494216}

In [10]:
for i in np.random.randint(0, 54, 5):
    query = f"*Your task is to classify the activity performed by the user based on the provided featurized accelerometer data.\n* The data is in the form of a TSFEL feature vector with features: {features}.\n* There are six possible activities: 1: WALKING, 2: WALKING_UPSTAIRS, 3: WALKING_DOWNSTAIRS, 4: SITTING, 5: STANDING, 6: LAYING\n\nWhat is this activity: {X_test_tsfel_df.loc[i].to_dict()}?"
    chat_completion = client.chat.completions.create(
        messages=[
            # Set an optional system message. This sets the behavior of the
            # assistant and can be used to provide specific instructions for
            # how it should behave throughout the conversation.
            {
                "role": "system",
                "content": "You are an activity classification model who will classify an activity based on provided TSFEL feature vector. Keep responses in the following format: 1: WALKING, 2: WALKING_UPSTAIRS, 3: WALKING_DOWNSTAIRS, 4: SITTING, 5: STANDING, 6: LAYING"
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": query,
            }
        ],
        # The language model which will generate the completion.
        model="llama-3.1-70b-versatile",

        # optional parameters

        # Controls randomness: lowering results in less random completions.
        temperature=0,
    )
    print(chat_completion.choices[0].message.content)
    print("--------------------------------")
    print(f"Actual Activity - {y_test[i]}: {label[y_test[i]]}")
    print("===============================================")

Based on the provided TSFEL feature vector, I would classify the activity as: 4: SITTING.

The reasoning behind this classification is as follows:

- The '0_Mean' and '0_Median' values are close to 1, indicating a relatively stable acceleration signal, which is consistent with sitting.
- The '0_Energy' value is very low, indicating a low level of movement, which is also consistent with sitting.
- The '0_Spectral entropy' value is close to 1, indicating a relatively low level of complexity in the signal, which is consistent with sitting.
- The '0_Wavelet absolute mean_8' and '0_Wavelet variance_8' values are relatively low, indicating a low level of movement and variability in the signal, which is consistent with sitting.

Overall, the feature vector suggests a relatively stable and low-movement activity, which is most consistent with sitting.
--------------------------------
Actual Activity - 4: SITTING
Based on the provided TSFEL feature vector, I would classify the activity as: 1: WA

Here the LLM gets 1 out of 5 correct.

Now lets test the LLM on 5 random samples from the test set and few Shot by giving it 10 random examples from the training set.

In [29]:
y_few_shot_pred_tup = []
for i in np.random.randint(0, 54, 5):
    query = f"*Your task is to classify the activity performed by the user based on the provided featurized accelerometer data. \n* The data is in the form of a TSFEL feature vector with features: {features} \n* There are six possible activities - 1: WALKING, 2: WALKING_UPSTAIRS, 3: WALKING_DOWNSTAIRS, 4: SITTING, 5: STANDING, 6: LAYING.\n* Please provide the most likely activity as a single integer corresponding to the activity.\n Here are few examples with values corresponding to the given features:"
    # giving it random 10 examples from the training data
    for j in np.random.randint(0, 126, 10):
        query+=f"{j+1}.\n"
        query+=f"TSFEL Feature vector = {list(X_train_tsfel_df.loc[j].to_dict().values())}\n"
        query+=f"Activity/Label = {y_train[j]}: {label[y_train[j]]}\n"
        query+="\n"
    query+=f"\nPredict the activity with the following feature vector: {X_test_tsfel_df.loc[i].to_dict().values()}."
    chat_completion = client.chat.completions.create(
        messages=[
            # Set an optional system message. This sets the behavior of the
            # assistant and can be used to provide specific instructions for
            # how it should behave throughout the conversation.
            {
                "role": "system",
                "content": f"You are an activity classification model. Keep responses in the following format: 1: WALKING, 2: WALKING_UPSTAIRS, 3: WALKING_DOWNSTAIRS, 4: SITTING, 5: STANDING, 6: LAYING. You should output a single integer corresponding to the activity label."
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": query,
            }
        ],
        # The language model which will generate the completion.
        model="llama-3.1-70b-versatile",
        # optional parameters

        # Controls randomness: lowering results in less random completions.
        temperature=0,
    )
    T.sleep(2)
    # append the completion returned by the LLM to y_pred
    print(chat_completion.choices[0].message.content)
    print("--------------------------------")
    print(f"Actual Activity - {y_test[i]}: {label[y_test[i]]}")
    print("===============================================")

1: WALKING
--------------------------------
Actual Activity - 1: WALKING
1: WALKING
--------------------------------
Actual Activity - 1: WALKING
4: SITTING
--------------------------------
Actual Activity - 5: STANDING
1: WALKING
--------------------------------
Actual Activity - 2: WALKING_UPSTAIRS
3: WALKING_DOWNSTAIRS
--------------------------------
Actual Activity - 3: WALKING_DOWNSTAIRS


In this case the LLM is correct 3 out of 5 times. Clearly few shot is better than zero shot here.
This is largerly because the LLM now has context and it can better understand the feature vectors and patterns and adapt it predictions on new data accordingly.

# Question 2

In [12]:
y_few_shot_pred_tup = []
for i in np.random.randint(0, 54, 27):
    query = f"*Your task is to classify the activity performed by the user based on the provided featurized accelerometer data.\n* The features are: {features}. \n* You are given values corresponding to the features in order. \n* There are six possible activities - 1: WALKING, 2: WALKING_UPSTAIRS, 3: WALKING_DOWNSTAIRS, 4: SITTING, 5: STANDING, 6: LAYING.\n* Please provide the most likely activity as a single integer corresponding to the activity.\n Here are few examples with values corresponding to the given features:"
    # giving it random 10 examples from the training data
    for j in np.random.randint(0, 126, 10):
        query+=f"{j+1}.\n"
        query+=f"Feature vector = {list(X_train_tsfel_df.loc[j].to_dict().values())}\n"
        query+=f"Activity = {y_train[j]}: {label[y_train[j]]}\n"
    query+=f"\nWhat is this activity: {list(X_test_tsfel_df.loc[i].to_dict().values())}?"
    chat_completion = client.chat.completions.create(
        messages=[
            # Set an optional system message. This sets the behavior of the
            # assistant and can be used to provide specific instructions for
            # how it should behave throughout the conversation.
            {
                "role": "system",
                "content": f"You are an activity classification model. Keep responses in the following format: 1: WALKING, 2: WALKING_UPSTAIRS, 3: WALKING_DOWNSTAIRS, 4: SITTING, 5: STANDING, 6: LAYING. You should output a single integer corresponding to the activity label."
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": query,
            }
        ],
        # The language model which will generate the completion.
        model="llama-3.1-70b-versatile",
        # optional parameters

        # Controls randomness: lowering results in less random completions.
        temperature=0,
    )
    T.sleep(2)
    # append the completion returned by the LLM to y_pred
    y_few_shot_pred_tup.append((i,chat_completion.choices[0].message.content))
    print(y_few_shot_pred_tup[-1])

(np.int64(50), '3: WALKING_DOWNSTAIRS')
(np.int64(46), '1: WALKING')
(np.int64(42), '1: WALKING')
(np.int64(39), '5: STANDING')
(np.int64(13), '6: LAYING')
(np.int64(50), '3: WALKING_DOWNSTAIRS')
(np.int64(39), '6: LAYING')
(np.int64(7), '5: STANDING')
(np.int64(10), '4: SITTING')
(np.int64(6), '2: WALKING_UPSTAIRS')
(np.int64(5), '1: WALKING')
(np.int64(9), '2: WALKING_UPSTAIRS')
(np.int64(5), '1: WALKING')
(np.int64(44), '4: SITTING')
(np.int64(46), '5: STANDING')
(np.int64(51), '1: WALKING')
(np.int64(19), '4: SITTING')
(np.int64(24), '1: WALKING')
(np.int64(16), '5: STANDING')
(np.int64(36), '4: SITTING')
(np.int64(2), '1: WALKING')
(np.int64(49), '3: WALKING_DOWNSTAIRS')
(np.int64(23), '1: WALKING')
(np.int64(32), '4: SITTING')
(np.int64(5), '1: WALKING')
(np.int64(33), '4: SITTING')
(np.int64(48), '3: WALKING_DOWNSTAIRS')


In [26]:
new_y_test = [int(y_test[int(x[0])]) for x in y_few_shot_pred_tup]
y_pred = [int(x[1][0]) for x in y_few_shot_pred_tup]

In [16]:
accuracy_score(new_y_test, y_pred)

0.5925925925925926

In [25]:
# Train a Decision Tree Classifier and get predictions
dtc_tsfel = DecisionTreeClassifier()
dtc_tsfel.fit(X_train_tsfel, y_train)
y_pred_tsfel = dtc_tsfel.predict(X_test_tsfel)
acc_dt = accuracy_score(y_pred_tsfel, y_test)
acc_dt

0.6666666666666666

Decision tree clearly has better accuracy than the LLM. This is because the LLM is a language model and is not trained to predict the labels of the data. It is trained to predict the next word in a sentence given the previous words. The decision tree on the other hand is trained to predict the labels of the data. This is why the decision tree has better accuracy than the LLM.

# Question 3

### Limitations of Zero-Shot Learning and Few-Shot Learning

- Since LLMs are large language models trained on text data, they may not be able to capture the complex temporal and spatial patterns present in sensor data, limiting their performance on activity recognition tasks. 
- Also they may not be able to differentiate between small variations in the data between two similar activities. 
- The patterns in the accelerometer data also depend on the individual and the environment, which may not be captured by the LLMs.
- It might be the case that the LLM is trained in a different domain and context and might not understand the patterns in the accelerometer data.

# Question 4

In [30]:
# chose a random number between 1,6
leave_out = np.random.randint(1,7)
new_y_train = []
valid_idx = []
for i in range(len(y_train)):
    if y_train[i] != leave_out:
        new_y_train.append(y_train[i])
        valid_idx.append(i)
print(leave_out)
invalid_idx = []
for i in range(len(y_test)):
    if y_test[i] == leave_out:
        invalid_idx.append(i)

5


In [34]:
print("Actual Left Out Activity Activity - ",leave_out,":",label[leave_out])
print("---")
for i in random.sample(invalid_idx, 5):
    query = f"*Your task is to classify the activity performed by the user based on the provided featurized accelerometer data.\n* The features are: {features}. \n* You are given values corresponding to the features in order. \n* There are six possible activities - 1: WALKING, 2: WALKING_UPSTAIRS, 3: WALKING_DOWNSTAIRS, 4: SITTING, 5: STANDING, 6: LAYING.\n* Please provide the most likely activity as a single integer corresponding to the activity.\n Here are few examples with values corresponding to the given features:"
    # giving it random 10 examples from the training data
    for j in random.sample(valid_idx, 10):
        query+=f"{j+1}.\n"
        query+=f"Feature vector = {list(X_train_tsfel_df.loc[j].to_dict().values())}\n"
        query+=f"Activity = {y_train[j]}: {label[y_train[j]]}\n"
    query+=f"\nWhat is this activity: {list(X_test_tsfel_df.loc[i].to_dict().values())}?"
    chat_completion = client.chat.completions.create(
        messages=[
            # Set an optional system message. This sets the behavior of the
            # assistant and can be used to provide specific instructions for
            # how it should behave throughout the conversation.
            {
                "role": "system",
                "content": f"You are an activity classification model. Keep responses in the following format: 1: WALKING, 2: WALKING_UPSTAIRS, 3: WALKING_DOWNSTAIRS, 4: SITTING, 5: STANDING, 6: LAYING. You should output a single integer corresponding to the activity label."
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": query,
            }
        ],
        # The language model which will generate the completion.
        model="llama-3.1-70b-versatile",
        # optional parameters

        # Controls randomness: lowering results in less random completions.
        temperature=0,
    )
    T.sleep(2)
    # append the completion returned by the LLM to y_pred
    print(f"Prediction - ",chat_completion.choices[0].message.content)
    print("===============================================")

Actual Left Out Activity Activity -  5 : STANDING
---
Prediction -  4: SITTING
Prediction -  4: SITTING
Prediction -  6: LAYING
Prediction -  4: SITTING
Prediction -  4: SITTING


The LLM is unable to classify the activity correctly because the LLM has not seen the activity before and does not have enough context to understand the patterns in the accelerometer data. The LLM is trained on a different domain and context and is not able to generalize to the unseen activity. The LLM is not able to differentiate between the unseen activity and the other activities in the dataset.

# Question 5

In [37]:
for i in np.random.randint(0, 54, 5):
    query = f"*Your task is to classify the activity performed by the user based on the provided featurized accelerometer data.\n* The features are: {features}. \n* You are given values corresponding to the features in order. \n* There are six possible activities - 1: WALKING, 2: WALKING_UPSTAIRS, 3: WALKING_DOWNSTAIRS, 4: SITTING, 5: STANDING, 6: LAYING.\n* Please provide the most likely activity as a single integer corresponding to the activity.\n Here are few examples with values corresponding to the given features:"
    # giving it random 10 examples from the training data
    for j in np.random.randint(0, 126, 10):
        query+=f"{j+1}.\n"
        query+=f"Feature vector = {list(X_train_tsfel_df.loc[j].to_dict().values())}\n"
        query+=f"Activity = {y_train[j]}: {label[y_train[j]]}\n"
    query+=f"\nWhat is this activity: {[float(random.uniform(X_train_tsfel_df[feature].min(), X_train_tsfel_df[feature].max())) for feature in features]}?"
    chat_completion = client.chat.completions.create(
        messages=[
            # Set an optional system message. This sets the behavior of the
            # assistant and can be used to provide specific instructions for
            # how it should behave throughout the conversation.
            {
                "role": "system",
                "content": f"You are an activity classification model. Keep responses in the following format: 1: WALKING, 2: WALKING_UPSTAIRS, 3: WALKING_DOWNSTAIRS, 4: SITTING, 5: STANDING, 6: LAYING. You should output a single integer corresponding to the activity label."
            },
            # Set a user message for the assistant to respond to.
            {
                "role": "user",
                "content": query,
            }
        ],
        # The language model which will generate the completion.
        model="llama-3.1-70b-versatile",
        # optional parameters

        # Controls randomness: lowering results in less random completions.
        temperature=0,
    )
    T.sleep(2)
    # append the completion returned by the LLM to y_pred
    print(f"LLM's Predicition: {chat_completion.choices[0].message.content}")
    print("===============================================")

LLM's Predicition: 2: WALKING_UPSTAIRS
LLM's Predicition: 4: SITTING
LLM's Predicition: 4: SITTING
LLM's Predicition: 2: WALKING_UPSTAIRS
LLM's Predicition: 1: WALKING


The LLM is making some predictions on the random feature vector it is fed. 